In [1]:
import pandas as pd


In [4]:
df = pd.read_csv('http://bit.ly/kaggletrain')

In [6]:
df.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [7]:
df.shape

(891, 12)

In [9]:
df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [10]:
df = df.loc[df.Embarked.notna(), ['Survived', 'Pclass', 'Sex', 'Embarked']]

In [11]:
df.head()

,Survived,Pclass,Sex,Embarked
0,0,3,male,S
1,1,1,female,C
2,1,3,female,S
3,1,1,female,S
4,0,3,male,S


In [12]:
df.shape

(889, 4)

In [13]:
X = df.loc[:,['Pclass']]
y = df.Survived

X.shape, y.shape

((889, 1), (889,))

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
logreg = LogisticRegression(solver='lbfgs')

In [17]:
from sklearn.model_selection import cross_val_score

In [19]:
cross_val_score(logreg, X, y, cv=5, scoring='accuracy').mean()

0.6783406335301212

In [20]:
y.value_counts(normalize=True)

0    0.617548
1    0.382452
Name: Survived, dtype: float64

In [21]:
from sklearn.preprocessing import OneHotEncoder

In [23]:
ohe = OneHotEncoder(sparse=False)

In [26]:
ohe.fit_transform(df[['Embarked']])

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [27]:
ohe.categories_

[array(['C', 'Q', 'S'], dtype=object)]

In [28]:
X = df.drop('Survived', axis='columns')
X.head()

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S


In [29]:
from sklearn.compose import make_column_transformer

In [31]:
column_trans = make_column_transformer((OneHotEncoder(),['Sex', 'Embarked']), remainder='passthrough')

In [32]:
column_trans.fit_transform(X)

array([[0., 1., 0., 0., 1., 3.],
       [1., 0., 1., 0., 0., 1.],
       [1., 0., 0., 0., 1., 3.],
       ...,
       [1., 0., 0., 0., 1., 3.],
       [0., 1., 1., 0., 0., 1.],
       [0., 1., 0., 1., 0., 3.]])

In [33]:
from sklearn.pipeline import make_pipeline

In [34]:
pipe = make_pipeline(column_trans, logreg)

In [36]:
cross_val_score(pipe, X, y, cv=5, scoring='accuracy').mean()

0.7727924839713071

In [37]:
X_new = X.sample(5, random_state=99)

In [38]:
X_new

,Pclass,Sex,Embarked
599,1,male,C
512,1,male,S
273,1,male,C
215,1,female,C
790,3,male,Q


In [39]:
pipe.fit(X,y)
X

,Pclass,Sex,Embarked
0,3,male,S
1,1,female,C
2,3,female,S
3,1,female,S
4,3,male,S
...,...,...,...
886,2,male,S
887,1,female,S
888,3,female,S
889,1,male,C


In [40]:
pipe.predict(X_new)

array([1, 0, 1, 1, 0])